In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin
from fairlearn.postprocessing import ThresholdOptimizer
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import warnings

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)

# 1. Caricamento e preprocessing COMPAS
df = pd.read_csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")
df = df[df['race'].isin(['Caucasian', 'African-American'])].copy()
df['race_binary'] = (df['race'] == 'African-American').astype(int)
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda x: 1 if x == 'F' else 0)

features = ['age', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'c_charge_degree']
X = df[features].fillna(0)
y = df['two_year_recid'].astype(int).values
sensitive_attr = df['race_binary'].values

X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=0, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)

In [3]:
# 2. Adversarial Debiasing Classico
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

class Adv(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

main_model = MLP(X.shape[1])
adversary = Adv()
optimizer_main = torch.optim.Adam(main_model.parameters(), lr=0.001)
optimizer_adv = torch.optim.Adam(adversary.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
sens_train_torch = torch.tensor(sens_train, dtype=torch.float32).view(-1, 1)

for epoch in range(1):
    for x, y_, s in zip(X_train_torch, y_train_torch, sens_train_torch):
        x = x.view(1, -1)
        y_ = y_.view(1, -1)
        s = s.view(1, -1)

        optimizer_main.zero_grad()
        optimizer_adv.zero_grad()

        pred = main_model(x)
        pred_loss = loss_fn(pred, y_)
        adv_pred = adversary(pred.detach())
        adv_loss = loss_fn(adv_pred, s)

        loss = pred_loss - adv_loss
        loss.backward()
        optimizer_main.step()
        optimizer_adv.step()

main_pred_scores = main_model(X_test_torch).detach().numpy().flatten().astype(np.float64)
main_pred = (main_pred_scores >= 0.5).astype(int)


In [4]:
# 3. Adversarial Debiasing + Equalized Odds
class MainProbWrapper(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - main_pred_scores, main_pred_scores]).T
    def _check_is_fitted(self):
        pass

X_dummy = np.zeros((len(y_test), 1))
postproc_adv = ThresholdOptimizer(
    estimator=MainProbWrapper(),
    constraints="equalized_odds",
    prefit=True
)
postproc_adv.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_adv_fair = postproc_adv.predict(X=X_dummy, sensitive_features=sens_test)


In [6]:
# 4. Adversarial Debiasing Quantistico
num_qubits = X.shape[1]
x_params = ParameterVector("x", num_qubits)
y_params = ParameterVector("y", num_qubits)
qc = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc.ry(x_params[i], i)
for _ in range(2):
    for i in range(num_qubits - 1):
        qc.cz(i, i+1)
    for i in range(num_qubits):
        qc.rx(y_params[i], i)
        qc.ry(y_params[i], i)

qnn = EstimatorQNN(circuit=qc, input_params=x_params, weight_params=y_params)
model = TorchConnector(qnn)

class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn = model
    def forward(self, x):
        return torch.sigmoid(self.qnn(x))

qnn_model = QuantumClassifier()
adversary_q = Adv()
optimizer_main_q = torch.optim.Adam(qnn_model.parameters(), lr=0.001)
optimizer_adv_q = torch.optim.Adam(adversary_q.parameters(), lr=0.001)

for epoch in range(1):
    for x, y_, s in zip(X_train_torch, y_train_torch, sens_train_torch):
        x = x.view(1, -1)
        y_ = y_.view(1, -1)
        s = s.view(1, -1)

        optimizer_main_q.zero_grad()
        optimizer_adv_q.zero_grad()

        pred = qnn_model(x)
        pred_loss = loss_fn(pred, y_)
        adv_pred = adversary_q(pred.detach())
        adv_loss = loss_fn(adv_pred, s)

        loss = pred_loss - adv_loss
        loss.backward()
        optimizer_main_q.step()
        optimizer_adv_q.step()

qnn_scores = qnn_model(X_test_torch).detach().numpy().flatten().astype(np.float64)
qnn_preds = (qnn_scores >= 0.5).astype(int)

class QNNProbWrapper(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - qnn_scores, qnn_scores]).T
    def _check_is_fitted(self):
        pass

postproc_qadv = ThresholdOptimizer(
    estimator=QNNProbWrapper(),
    constraints="equalized_odds",
    prefit=True
)
postproc_qadv.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_qadv_fair = postproc_qadv.predict(X=X_dummy, sensitive_features=sens_test)

# Metriche

def full_metrics(y_true, y_pred, y_score, sensitive_attr):
    s = np.array(sensitive_attr)
    yt, yp = np.array(y_true), np.array(y_pred)
    priv, unpriv = s == 0, s == 1

    def tpr(y, yhat): return np.mean((yhat == 1) & (y == 1)) / max(np.mean(y == 1), 1e-6)
    def fpr(y, yhat): return np.mean((yhat == 1) & (y == 0)) / max(np.mean(y == 0), 1e-6)
    sr_priv, sr_unpriv = np.mean(yp[priv]), np.mean(yp[unpriv])

    return {
        'Accuracy': accuracy_score(yt, yp),
        'AUC': roc_auc_score(yt, y_score),
        'F1': f1_score(yt, yp),
        'Precision': precision_score(yt, yp, zero_division=0),
        'Recall': recall_score(yt, yp),
        'MCC': matthews_corrcoef(yt, yp),
        'EOD': tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]),
        'AOD': 0.5 * ((fpr(yt[unpriv], yp[unpriv]) - fpr(yt[priv], yp[priv])) +
                      (tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]))),
        'DI': sr_unpriv / sr_priv if sr_priv > 0 else np.nan,
        'SPR': sr_unpriv - sr_priv
    }

results = pd.DataFrame(columns=[
    'Accuracy', 'AUC', 'F1', 'Precision', 'Recall', 'MCC', 'EOD', 'AOD', 'DI', 'SPR'
])
results.loc['Adversarial Classic'] = full_metrics(y_test, main_pred, main_pred_scores, sens_test)
results.loc['Adv + EQ Odds'] = full_metrics(y_test, y_pred_adv_fair, main_pred_scores, sens_test)
results.loc['Quantum Adv'] = full_metrics(y_test, qnn_preds, qnn_scores, sens_test)
results.loc['QAdv + EQ Odds'] = full_metrics(y_test, y_pred_qadv_fair, qnn_scores, sens_test)
print(results)


                     Accuracy       AUC        F1  Precision    Recall  \
Adversarial Classic  0.670461  0.729229  0.646512   0.646512  0.646512   
Adv + EQ Odds        0.668293  0.729229  0.565341   0.726277  0.462791   
Quantum Adv          0.528997  0.551283  0.203483   0.480519  0.129070   
QAdv + EQ Odds       0.527913  0.551283  0.564282   0.495171  0.655814   

                          MCC       EOD       AOD        DI       SPR  
Adversarial Classic  0.337882  0.260833  0.239524  1.868789  0.265297  
Adv + EQ Odds        0.338983 -0.011071 -0.009617  1.062939  0.018007  
Quantum Adv          0.010917  0.039524  0.005335  1.018754  0.002322  
QAdv + EQ Odds       0.073958  0.003810  0.010718  1.029223  0.017727  
